In [26]:
from pandas_datareader import data as pdr
import yfinance as yf
import plotly.graph_objects as go

yf.pdr_override()

# download dataframe
data = pdr.get_data_yahoo(tickers = "AMZN AAPL MSFT GOOGL NFLX FB"
                          , start="2019-01-01"
                          , end="2020-11-14")

data.head()

[*********************100%***********************]  6 of 6 completed


Adj Close                                                    \
                 AAPL         AMZN          FB        GOOGL        MSFT   
Date                                                                      
2019-01-02  38.249401  1539.130005  135.679993  1054.680054   98.860214   
2019-01-03  34.439476  1500.280029  131.740005  1025.469971   95.223351   
2019-01-04  35.909672  1575.390015  137.949997  1078.069946   99.652115   
2019-01-07  35.829746  1629.510010  138.050003  1075.920044   99.779205   
2019-01-08  36.512772  1656.579956  142.529999  1085.369995  100.502670   

                            Close                                        ...  \
                  NFLX       AAPL         AMZN          FB        GOOGL  ...   
Date                                                                     ...   
2019-01-02  267.660004  39.480000  1539.130005  135.679993  1054.680054  ...   
2019-01-03  271.200012  35.547501  1500.280029  131.740005  1025.469971  ...   
2019-01-04  297.570007  37.064999  1575.390015  137.949997  1078.069946  ...   
2019-01-07  315.339996  36.982498  1629.510010  138.050003  1075.920044  ...   
2019-01-08  320.269989  37.687500  1656.579956  142.529999  1085.369995  ...   

                  Open                                          Volume  \
                    FB        GOOGL        MSFT        NFLX       AAPL   
Date                                                                     
2019-01-02  128.990005  1027.199951   99.550003  259.279999  148158800   
2019-01-03  134.690002  1050.670044  100.099998  270.200012  365248800   
2019-01-04  134.009995  1042.560059   99.720001  281.880005  234428400   
2019-01-07  137.559998  1080.969971  101.639999  302.100006  219111200   
2019-01-08  139.889999  1086.000000  103.040001  319.980011  164101200   

                                                            
               AMZN        FB    GOOGL      MSFT      NFLX  
Date                                                        
2019-01-02  7983100  28146200  1593400  35329300  11679500  
2019-01-03  6975600  22717900  2098000  42579100  14969600  
2019-01-04  9182600  29002100  2301100  44060600  19330100  
2019-01-07  7993200  20089300  2372300  35656100  18620100  
2019-01-08  8881400  26263800  1770700  31514400  15359200  

[5 rows x 36 columns]

In [27]:
def computeRSI (data, time_window):
    diff = data.diff(1).dropna() # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg = up_chg.ewm(com=time_window-1, min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1, min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

In [32]:
close = data['Adj Close']
close = close.reset_index()

close['RSI_APPLE'] = computeRSI(close['AAPL'],14)
close['RSI_AMZN'] = computeRSI(close['AMZN'],14)
close['RSI_FB'] = computeRSI(close['FB'],14)
close['RSI_GOOGL'] = computeRSI(close['GOOGL'],14)
close['RSI_MSFT'] = computeRSI(close['MSFT'],14)
close['RSI_NFLX'] = computeRSI(close['NFLX'],14)
close['low'] = 30
close['high'] = 70
close.head(20)

,Date,AAPL,AMZN,FB,GOOGL,MSFT,NFLX,RSI_APPLE,RSI_AMZN,RSI_FB,RSI_GOOGL,RSI_MSFT,RSI_NFLX,low,high
0,2019-01-02,38.249401,1539.130005,135.679993,1054.680054,98.860214,267.660004,NaN,NaN,NaN,NaN,NaN,NaN,30,70
1,2019-01-03,34.439476,1500.280029,131.740005,1025.469971,95.223351,271.200012,NaN,NaN,NaN,NaN,NaN,NaN,30,70
2,2019-01-04,35.909672,1575.390015,137.949997,1078.069946,99.652115,297.570007,NaN,NaN,NaN,NaN,NaN,NaN,30,70
3,2019-01-07,35.829746,1629.510010,138.050003,1075.920044,99.779205,315.339996,NaN,NaN,NaN,NaN,NaN,NaN,30,70
4,2019-01-08,36.512772,1656.579956,142.529999,1085.369995,100.502670,320.269989,NaN,NaN,NaN,NaN,NaN,NaN,30,70
5,2019-01-09,37.132820,1659.420044,144.229996,1081.650024,101.939812,319.959991,NaN,NaN,NaN,NaN,NaN,NaN,30,70
6,2019-01-10,37.251511,1656.219971,144.199997,1078.829956,101.284798,324.660004,NaN,NaN,NaN,NaN,NaN,NaN,30,70
7,2019-01-11,36.885773,1640.560059,143.800003,1064.469971,100.502670,337.589996,NaN,NaN,NaN,NaN,NaN,NaN,30,70
8,2019-01-14,36.331120,1617.209961,145.389999,1051.510010,99.769432,332.940002,NaN,NaN,NaN,NaN,NaN,NaN,30,70
9,2019-01-15,37.074699,1674.560059,148.949997,1086.510010,102.663292,354.640015,NaN,NaN,NaN,NaN,NaN,NaN,30,70


In [46]:
# print(close.tail())

#plot
fig = go.Figure()

#create lines/traces
fig.add_trace(go.Scatter(x=close['Date'], y=close['RSI_APPLE'],
                    mode='lines',
                    name='Apple',
                    line=dict(color="Silver", width=1),))
fig.add_trace(go.Scatter(x=close['Date'], y=close['RSI_AMZN'],
                    mode='lines',
                    name='Amazon',
                    line=dict(color="orange", width=1),))
fig.add_trace(go.Scatter(x=close['Date'], y=close['RSI_FB'],
                    mode='lines',
                    name='FaceBook',
                    line=dict(color="royalblue", width=1),))
fig.add_trace(go.Scatter(x=close['Date'], y=close['RSI_GOOGL'],
                    mode='lines',
                    name='Google',
                    line=dict(color="LightGreen", width=1),))
fig.add_trace(go.Scatter(x=close['Date'], y=close['RSI_MSFT'],
                    mode='lines',
                    name='Microsoft',
                    line=dict(color="MediumPurple", width=1),))
fig.add_trace(go.Scatter(x=close['Date'], y=close['RSI_NFLX'],
                    mode='lines',
                    name='Netflix',
                    line=dict(color="Red", width=1),))

fig.add_trace(go.Scatter(x=close['Date'], y=close['high'],
                         fill=None,
                         mode='lines',
                         line=dict(width=0.5, color='rgb(222, 196, 255)')))
fig.add_trace(go.Scatter(x=close['Date'],y=close['low'],
                         fill='tonexty', # fill area between trace0 and trace1
                         mode='lines',
                         line=dict(width=0.5, color='rgb(222, 196, 255)')))

#update axis ticks
fig.update_yaxes(nticks=30,showgrid=True)
fig.update_xaxes(nticks=12,showgrid=True)

#update layout
fig.update_layout(title="<b>Daily RSI</b>"
                 , height = 700
                 , xaxis_title='Date'
                 , yaxis_title='Relative Strength Index'
                 , template = "plotly_white" #['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark']
                 )

#update legend
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.show()

In [24]:
close = data['Adj Close']
# Get the difference in price from previous step
delta = close.diff()
# Get rid of the first row, which is NaN since it did not have a previous 
delta = delta[1:] 

# Make the positive gains (up) and negative gains (down) Series
up, down = delta.copy(), delta.copy()
up[up < 0] = 0
down[down > 0] = 0

# Calculate the EWMA
roll_up1 = up.ewm(span=window_length).mean()
roll_down1 = down.abs().ewm(span=window_length).mean()

# Calculate the RSI based on EWMA
RS1 = roll_up1 / roll_down1
RSI1 = 100.0 - (100.0 / (1.0 + RS1))

# Calculate the SMA
roll_up2 = up.rolling(window_length).mean()
roll_down2 = down.abs().rolling(window_length).mean()

# Calculate the RSI based on SMA
RS2 = roll_up2 / roll_down2
RSI2 = 100.0 - (100.0 / (1.0 + RS2))

delta3 = data.Close.diff()
window3 = 15
up_days3 = delta3.copy()
up_days3[delta3<=0]=0.0
down_days3 = abs(delta3.copy())
down_days3[delta3>0]=0.0
RS_up3 = up_days3.rolling(window3).mean()
RS_down3 = down_days3.rolling(window3).mean()
RSI3 = 100-100/(1+RS_up3/RS_down3)

#Reset Indexes
RSI1 = RSI1.reset_index()
RSI2 = RSI2.reset_index()
RSI3 = RSI3.reset_index()

# Compare graphically
# plt.figure(figsize=(8, 6))
# RSI1.plot()
# RSI2.plot()
# plt.legend(['RSI via EWMA', 'RSI via SMA'])
# plt.show()

NameError: name 'window_length' is not defined

In [104]:
print(RSI1.tail())

#plot
fig = go.Figure()

#create lines/traces
fig.add_trace(go.Scatter(x=RSI1['Date'], y=RSI1['AAPL'],
                    mode='lines',
                    name='Apple',
                    line=dict(color="Silver", width=1),))
fig.add_trace(go.Scatter(x=RSI1['Date'], y=RSI1['AMZN'],
                    mode='lines',
                    name='Amazon',
                    line=dict(color="orange", width=1),))
fig.add_trace(go.Scatter(x=RSI1['Date'], y=RSI1['FB'],
                    mode='lines',
                    name='FaceBook',
                    line=dict(color="royalblue", width=1),))
fig.add_trace(go.Scatter(x=RSI1['Date'], y=RSI1['GOOGL'],
                    mode='lines',
                    name='Google',
                    line=dict(color="LightGreen", width=1),))
fig.add_trace(go.Scatter(x=RSI1['Date'], y=RSI1['MSFT'],
                    mode='lines',
                    name='Microsoft',
                    line=dict(color="MediumPurple", width=1),))
fig.add_trace(go.Scatter(x=RSI1['Date'], y=RSI1['NFLX'],
                    mode='lines',
                    name='Netflix',
                    line=dict(color="Red", width=1),))

#update axis ticks
fig.update_yaxes(nticks=20)
fig.update_xaxes(nticks=12)

#update layout
fig.update_layout(title="<b>RSI via EWMA</b>"
                 , height = 700
                 , xaxis_title='Date'
                 , yaxis_title='Relative Strength Index'
                 , template = "plotly" #['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark']
                 )

#update legend
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.show()

          Date       AAPL       AMZN         FB      GOOGL       MSFT  \
467 2020-11-09  53.856524  45.156577  50.745168  76.870282  58.357567   
468 2020-11-10  52.709488  37.637440  45.755335  68.350598  45.527505   
469 2020-11-11  62.081172  47.191008  49.421194  69.895414  54.241293   
470 2020-11-12  60.972158  45.080749  48.123994  68.116430  52.306880   
471 2020-11-13  61.115292  46.951626  50.140955  73.342624  54.126523   

          NFLX  
467  35.426859  
468  41.653499  
469  47.912966  
470  45.764361  
471  43.545106  


In [77]:
print(RSI2.tail())

#plot
fig = go.Figure()

#create lines/traces
fig.add_trace(go.Scatter(x=RSI2['Date'], y=RSI2['AAPL'],
                    mode='lines',
                    name='Apple',
                    line=dict(color="Silver", width=1),))
fig.add_trace(go.Scatter(x=RSI2['Date'], y=RSI2['AMZN'],
                    mode='lines',
                    name='Amazon',
                    line=dict(color="orange", width=1),))
fig.add_trace(go.Scatter(x=RSI2['Date'], y=RSI2['FB'],
                    mode='lines',
                    name='FaceBook',
                    line=dict(color="royalblue", width=1),))
fig.add_trace(go.Scatter(x=RSI2['Date'], y=RSI2['GOOGL'],
                    mode='lines',
                    name='Google',
                    line=dict(color="LightGreen", width=1),))
fig.add_trace(go.Scatter(x=RSI2['Date'], y=RSI2['MSFT'],
                    mode='lines',
                    name='Microsoft',
                    line=dict(color="MediumPurple", width=1),))
fig.add_trace(go.Scatter(x=RSI2['Date'], y=RSI2['NFLX'],
                    mode='lines',
                    name='Netflix',
                    line=dict(color="Red", width=1),))

#update axis ticks
fig.update_yaxes(nticks=20)
fig.update_xaxes(nticks=12)

#update layout
fig.update_layout(title="<b>RSI via SMA</b>"
                 , height = 700
                 , xaxis_title='Date'
                 , yaxis_title='Relative Strength Index'
                 , template = "plotly" #['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark']
                 )

#update legend
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.show()

          Date       AAPL       AMZN         FB      GOOGL       MSFT  \
467 2020-11-09  48.495069  46.424563  54.318509  71.488713  53.553786   
468 2020-11-10  48.925829  43.193380  47.479187  65.864368  46.833744   
469 2020-11-11  55.639616  48.367932  49.361377  65.048707  51.271059   
470 2020-11-12  56.331176  46.059358  46.057006  62.337687  49.392967   
471 2020-11-13  56.381782  46.767650  49.931822  72.067894  55.359219   

          NFLX  
467  34.426044  
468  47.057251  
469  51.768016  
470  49.520114  
471  48.325272  


In [23]:
print(RSI3.tail())

#plot
fig = go.Figure()

#create lines/traces
fig.add_trace(go.Scatter(x=RSI3['Date'], y=RSI3['AAPL'],
                    mode='lines',
                    name='Apple',
                    line=dict(color="Silver", width=1),))
fig.add_trace(go.Scatter(x=RSI3['Date'], y=RSI3['AMZN'],
                    mode='lines',
                    name='Amazon',
                    line=dict(color="orange", width=1),))
fig.add_trace(go.Scatter(x=RSI3['Date'], y=RSI3['FB'],
                    mode='lines',
                    name='FaceBook',
                    line=dict(color="royalblue", width=1),))
fig.add_trace(go.Scatter(x=RSI3['Date'], y=RSI3['GOOGL'],
                    mode='lines',
                    name='Google',
                    line=dict(color="LightGreen", width=1),))
fig.add_trace(go.Scatter(x=RSI3['Date'], y=RSI3['MSFT'],
                    mode='lines',
                    name='Microsoft',
                    line=dict(color="MediumPurple", width=1),))
fig.add_trace(go.Scatter(x=RSI3['Date'], y=RSI3['NFLX'],
                    mode='lines',
                    name='Netflix',
                    line=dict(color="Red", width=1),))

#update axis ticks
fig.update_yaxes(nticks=20)
fig.update_xaxes(nticks=12)

#update layout
fig.update_layout(title="<b>RSI via EWMA</b>"
                 , height = 700
                 , xaxis_title='Date'
                 , yaxis_title='Relative Strength Index'
                 , template = "plotly" #['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark']
                 )

#update legend
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.show()

NameError: name 'RSI3' is not defined

In [20]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "SPY AAPL MSFT",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "ytd",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1m",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

,AAPL,AMZN,FB,GOOGL,MSFT,NFLX
Date,,,,,,
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-09,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2020-11-09,48.495069,46.424563,54.318509,71.488713,53.553786,34.426044
2020-11-10,48.925829,43.193380,47.479187,65.864368,46.833744,47.057251
2020-11-11,55.639616,48.367932,49.361377,65.048707,51.271059,51.768016
